In [5]:
!pip install transformers -U --quiet

In [4]:
!pip install datasets transformers -U --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

In [7]:
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling
from datasets import load_dataset
import collections
import numpy as np
from transformers import default_data_collator
from transformers import DataCollatorForLanguageModeling

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

#Train WordPiece Tokenizer


In [ ]:
base_model = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(base_model)

dataset = load_dataset("RohanAiLab/persian_daily_news")

def get_training_corpus(dataset, chunk_size=1000):
    dataset = dataset["train"]
    for start_idx in range(0, len(dataset), chunk_size):
        samples = dataset[start_idx : start_idx + chunk_size]
        yield samples["text"]

training_corpus = get_training_corpus(dataset)
tokenizer = tokenizer.train_new_from_iterator(training_corpus, 128000)
tokenizer.push_to_hub("shekar-ai/bert_tokenzier_Persian")


CommitInfo(commit_url='https://huggingface.co/shekar-ai/bert_tokenzier_Persian/commit/d498466afa0d87e1096f4a1aa27a6f76199fff82', commit_message='Upload tokenizer', commit_description='', oid='d498466afa0d87e1096f4a1aa27a6f76199fff82', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shekar-ai/bert_tokenzier_Persian', endpoint='https://huggingface.co', repo_type='model', repo_id='shekar-ai/bert_tokenzier_Persian'), pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("shekar-ai/bert_tokenzier_Persian")

CommitInfo(commit_url='https://huggingface.co/shekar-ai/bert_tokenzier_Persian/commit/912484344924d9196eb1bf2b3c9b082a97d931a0', commit_message='Upload tokenizer', commit_description='', oid='912484344924d9196eb1bf2b3c9b082a97d931a0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shekar-ai/bert_tokenzier_Persian', endpoint='https://huggingface.co', repo_type='model', repo_id='shekar-ai/bert_tokenzier_Persian'), pr_revision=None, pr_num=None)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("shekar-ai/bert_tokenzier_Persian")


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.75M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

#Preprocessing Data

In [11]:
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForMaskedLM,Trainer,DataCollatorForLanguageModeling,  default_data_collator, DataCollatorForLanguageModeling
from datasets import load_dataset
import collections
import numpy as np

import torch._dynamo
torch._dynamo.config.suppress_errors = True

dataset = load_dataset("RohanAiLab/persian_blog", split='train[:2%]')
#dataset = load_dataset("RohanAiLab/persian_blog")

base_model = "google-bert/bert-base-uncased"

model = AutoModelForMaskedLM.from_pretrained(base_model)

model.resize_token_embeddings(len(tokenizer))

chunk_size = 128

def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=['text']
)

lm_datasets = tokenized_datasets.map(group_texts, batched=True)

wwm_probability = 0.2

def whole_word_masking_data_collator(features):
    for feature in features:

        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)




Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/7718 [00:00<?, ? examples/s]

Map:   0%|          | 0/7718 [00:00<?, ? examples/s]

KeyError: "Column train not in the dataset. Current columns in the dataset: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels']"

In [13]:
train_size = int(0.9 * len(lm_datasets))
test_size = int(0.1 * len(lm_datasets))

split_dataset = lm_datasets.train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)


In [20]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59496
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 6610
    })
})

In [23]:
for i in range(5):
    print(f"Sample {i+1}:")
    print("labels", split_dataset['train'][i]["labels"])
    print("input ids:", split_dataset['train'][i]["input_ids"])
    print("ATT MASK:", split_dataset['train'][i]["attention_mask"])

    print("---")


Sample 1:
labels [395, 1078, 204, 197, 3672, 820, 100, 2296, 48205, 80002, 267, 209, 4679, 7548, 72, 187, 12729, 204, 6, 9164, 193, 3457, 100, 9517, 1902, 209, 32028, 665, 6, 1452, 100, 105055, 7319, 320, 2296, 48205, 80002, 543, 241, 18, 905, 204, 187, 1973, 7319, 267, 3374, 703, 27485, 200, 18, 703, 881, 2149, 36126, 105878, 183, 79998, 186, 204, 7368, 193, 2576, 30183, 123, 368, 320, 543, 7928, 2576, 543, 241, 18, 3, 2, 905, 15502, 345, 9100, 18500, 190, 73, 187, 17877, 345, 9100, 12, 412, 195, 354, 13, 1048, 19606, 1936, 100, 1204, 782, 6529, 429, 820, 200, 18, 187, 17877, 538, 195, 354, 72, 29289, 72, 1048, 19606, 820, 100, 1204, 782, 6529, 429, 1936, 200, 18, 197, 396, 1302, 441, 22207, 1984, 981]
input ids: [395, 1078, 204, 197, 3672, 820, 100, 2296, 48205, 80002, 267, 209, 4679, 7548, 72, 187, 12729, 204, 6, 9164, 193, 3457, 100, 9517, 1902, 209, 32028, 665, 6, 1452, 100, 105055, 7319, 320, 2296, 48205, 80002, 543, 241, 18, 905, 204, 187, 1973, 7319, 267, 3374, 703, 27485, 200,

In [ ]:

from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(split_dataset["train"]) // batch_size
model_name = base_model.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-newsfa",
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
    remove_unused_columns=False,
    hub_model_id="shekar-ai/BERT-base-Persian",
    report_to="none",
    save_steps = 10000,
    save_strategy="best",
    save_total_limit = 3,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=whole_word_masking_data_collator,
    tokenizer=tokenizer,
)


Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/385900 [00:00<?, ? examples/s]

Map:   0%|          | 0/385900 [00:00<?, ? examples/s]

<ipython-input-10-6cb429fc7140>:120: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.327600,4.617504


TrainOutput(global_step=30662, training_loss=5.327629815123955, metrics={'train_runtime': 5044.8416, 'train_samples_per_second': 388.973, 'train_steps_per_second': 6.078, 'total_flos': 1.2926893428867072e+17, 'train_loss': 5.327629815123955, 'epoch': 1.0})

#Before Pretraining

In [2]:
from transformers import pipeline

model_id = "google-bert/bert-base-uncased"
fill_mask = pipeline("fill-mask", model=model_id, tokenizer=model_id)
masked_sentence = "سلام، من [MASK] هستم."  # "Hello, I am [MASK]."
predictions = fill_mask(masked_sentence)

for prediction in predictions:
    print(f"Word: {prediction['token_str']}, Score: {prediction['score']}")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


Word: ##ت, Score: 0.16890378296375275
Word: ##ا, Score: 0.08093248307704926
Word: ##د, Score: 0.08009195327758789
Word: ##ي, Score: 0.05841268226504326
Word: ##ج, Score: 0.04654987156391144


#After Pretraining on a Persian Corpus

In [1]:
from transformers import pipeline

model_id = "shekar-ai/BERT-base-Persian"
fill_mask = pipeline("fill-mask", model=model_id, tokenizer=model_id)
masked_sentence = "سلام، من [MASK] هستم."  # "Hello, I am [MASK]."
predictions = fill_mask(masked_sentence)

for prediction in predictions:
    print(f"Word: {prediction['token_str']}, Score: {prediction['score']}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.75M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0


Word: هم, Score: 0.01923883520066738
Word: من, Score: 0.014948173426091671
Word: چی, Score: 0.014675114303827286
Word: اینجا, Score: 0.012969208881258965
Word: ،, Score: 0.012910923920571804
